# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Imports

In [2]:
import torch
from ocpmodels.trainers import ForcesTrainer
from ocpmodels import models

In [3]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

True


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Download and preprocess data

Please download the train S2EF 200K dataset from [here](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md#download-the-datasets). This example assumes the --get-edges flag to be present at preprocessing time.

In [4]:
# set the path to your local lmdb directory
train_src = "/home/jovyan/projects/ocp/data/s2ef/200k/train-demo"

## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [17]:
task = {
    'dataset': 'trajectory_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [18]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

**Optimizer**

In [19]:
optimizer = {
    'batch_size': 16, # if hitting GPU memory issues, lower this
    'eval_batch_size': 8,
    'num_workers': 8,
    'lr_initial': 0.0001,
    'lr_gamma': 0.1,
    'lr_milestones': [15, 20],
    'warmup_epochs': 10,
    'warmup_factor': 0.2,
    'max_epochs': 1, # used for demonstration purposes
    'force_coefficient': 100,
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `stds` need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [8]:
dataset = [
{'src': train_src, 'normalize_labels': False}, # train set 
{'src': train_src}, # val set (optional)
{'src': train_src} # test set (optional - writes predictions to disk)
]

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [9]:
trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

amp: false
cmd:
  checkpoint_dir: checkpoints/2021-01-06-16-49-04-SchNet-example
  identifier: SchNet-example
  logs_dir: logs/tensorboard/2021-01-06-16-49-04-SchNet-example
  print_every: 10
  results_dir: results/2021-01-06-16-49-04-SchNet-example
  seed: 0
  timestamp: 2021-01-06-16-49-04-SchNet-example
dataset:
  normalize_labels: false
  src: /home/jovyan/projects/ocp/data/s2ef/200k/train-demo
logger: tensorboard
logger_entity: null
logger_project: null
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  force_coefficient: 100
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 15
  - 20
  max_epochs: 1
  num_workers: 64
  warmup_epochs: 10
  warmup_factor: 0.2
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential e

## Check the model

In [11]:
print(trainer.model)

OCPDataParallel(
  (module): SchNet(hidden_channels=1024, num_filters=256, num_interactions=3, num_gaussians=200, cutoff=6.0)
)


## Train

In [12]:
trainer.train()

forcesx_mae: 0.7450, forcesy_mae: 0.8024, forcesz_mae: 0.8352, forces_mae: 0.7942, forces_cos: 0.0523, forces_magnitude: 1.4755, energy_mae: 34.4331, energy_force_within_threshold: 0.0000, loss: 113.8546, epoch: 0.0003
forcesx_mae: 0.5871, forcesy_mae: 0.6545, forcesz_mae: 0.6278, forces_mae: 0.6232, forces_cos: 0.0263, forces_magnitude: 1.1090, energy_mae: 32.7792, energy_force_within_threshold: 0.0000, loss: 95.0260, epoch: 0.0035
forcesx_mae: 0.4987, forcesy_mae: 0.5432, forcesz_mae: 0.5586, forces_mae: 0.5335, forces_cos: 0.0030, forces_magnitude: 0.9548, energy_mae: 23.5220, energy_force_within_threshold: 0.0000, loss: 76.8902, epoch: 0.0067
forcesx_mae: 0.3893, forcesy_mae: 0.4586, forcesz_mae: 0.4432, forces_mae: 0.4303, forces_cos: -0.0035, forces_magnitude: 0.7461, energy_mae: 19.4026, energy_force_within_threshold: 0.0000, loss: 62.5090, epoch: 0.0099
forcesx_mae: 0.3424, forcesy_mae: 0.4016, forcesz_mae: 0.4249, forces_mae: 0.3896, forces_cos: 0.0140, forces_magnitude: 0.654

forcesx_mae: 0.1572, forcesy_mae: 0.1844, forcesz_mae: 0.1891, forces_mae: 0.1769, forces_cos: 0.0481, forces_magnitude: 0.2624, energy_mae: 3.4449, energy_force_within_threshold: 0.0000, loss: 21.1611, epoch: 0.1219
forcesx_mae: 0.1572, forcesy_mae: 0.1776, forcesz_mae: 0.1807, forces_mae: 0.1719, forces_cos: 0.0335, forces_magnitude: 0.2534, energy_mae: 3.0279, energy_force_within_threshold: 0.0000, loss: 20.1898, epoch: 0.1251
forcesx_mae: 0.1717, forcesy_mae: 0.1934, forcesz_mae: 0.2013, forces_mae: 0.1888, forces_cos: 0.0335, forces_magnitude: 0.2845, energy_mae: 3.6823, energy_force_within_threshold: 0.0000, loss: 22.6440, epoch: 0.1283
forcesx_mae: 0.1627, forcesy_mae: 0.1934, forcesz_mae: 0.2017, forces_mae: 0.1859, forces_cos: 0.0414, forces_magnitude: 0.2795, energy_mae: 3.6687, energy_force_within_threshold: 0.0000, loss: 22.2938, epoch: 0.1315
forcesx_mae: 0.1689, forcesy_mae: 0.2036, forcesz_mae: 0.1978, forces_mae: 0.1901, forces_cos: 0.0374, forces_magnitude: 0.2884, ene

forcesx_mae: 0.1680, forcesy_mae: 0.1768, forcesz_mae: 0.1716, forces_mae: 0.1721, forces_cos: 0.0381, forces_magnitude: 0.2594, energy_mae: 3.2181, energy_force_within_threshold: 0.0000, loss: 20.5549, epoch: 0.2435
forcesx_mae: 0.1275, forcesy_mae: 0.1469, forcesz_mae: 0.1566, forces_mae: 0.1437, forces_cos: 0.0293, forces_magnitude: 0.2094, energy_mae: 3.1564, energy_force_within_threshold: 0.0000, loss: 17.5119, epoch: 0.2467
forcesx_mae: 0.1334, forcesy_mae: 0.1614, forcesz_mae: 0.1614, forces_mae: 0.1521, forces_cos: 0.0446, forces_magnitude: 0.2214, energy_mae: 3.0059, energy_force_within_threshold: 0.0000, loss: 18.1482, epoch: 0.2499
forcesx_mae: 0.1482, forcesy_mae: 0.1669, forcesz_mae: 0.1801, forces_mae: 0.1650, forces_cos: 0.0295, forces_magnitude: 0.2427, energy_mae: 3.1646, energy_force_within_threshold: 0.0000, loss: 19.4738, epoch: 0.2531
forcesx_mae: 0.1390, forcesy_mae: 0.1679, forcesz_mae: 0.1772, forces_mae: 0.1614, forces_cos: 0.0364, forces_magnitude: 0.2357, ene

forcesx_mae: 0.1246, forcesy_mae: 0.1478, forcesz_mae: 0.1496, forces_mae: 0.1406, forces_cos: 0.0297, forces_magnitude: 0.2039, energy_mae: 3.5981, energy_force_within_threshold: 0.0000, loss: 17.6697, epoch: 0.3651
forcesx_mae: 0.1079, forcesy_mae: 0.1219, forcesz_mae: 0.1235, forces_mae: 0.1178, forces_cos: 0.0413, forces_magnitude: 0.1629, energy_mae: 3.3930, energy_force_within_threshold: 0.0000, loss: 15.1729, epoch: 0.3683
forcesx_mae: 0.1275, forcesy_mae: 0.1389, forcesz_mae: 0.1473, forces_mae: 0.1379, forces_cos: 0.0430, forces_magnitude: 0.1889, energy_mae: 3.6207, energy_force_within_threshold: 0.0000, loss: 17.3980, epoch: 0.3715
forcesx_mae: 0.1220, forcesy_mae: 0.1508, forcesz_mae: 0.1493, forces_mae: 0.1407, forces_cos: 0.0458, forces_magnitude: 0.1933, energy_mae: 3.4300, energy_force_within_threshold: 0.0000, loss: 17.3409, epoch: 0.3747
forcesx_mae: 0.1156, forcesy_mae: 0.1382, forcesz_mae: 0.1418, forces_mae: 0.1319, forces_cos: 0.0160, forces_magnitude: 0.1881, ene

forcesx_mae: 0.1069, forcesy_mae: 0.1163, forcesz_mae: 0.1190, forces_mae: 0.1141, forces_cos: 0.0367, forces_magnitude: 0.1587, energy_mae: 3.4257, energy_force_within_threshold: 0.0000, loss: 14.8809, epoch: 0.4867
forcesx_mae: 0.1068, forcesy_mae: 0.1461, forcesz_mae: 0.1417, forces_mae: 0.1316, forces_cos: 0.0375, forces_magnitude: 0.2006, energy_mae: 3.5625, energy_force_within_threshold: 0.0000, loss: 16.7261, epoch: 0.4899
forcesx_mae: 0.0971, forcesy_mae: 0.1192, forcesz_mae: 0.1329, forces_mae: 0.1164, forces_cos: 0.0240, forces_magnitude: 0.1689, energy_mae: 2.8136, energy_force_within_threshold: 0.0000, loss: 14.4461, epoch: 0.4931
forcesx_mae: 0.1010, forcesy_mae: 0.1302, forcesz_mae: 0.1387, forces_mae: 0.1233, forces_cos: 0.0503, forces_magnitude: 0.1742, energy_mae: 3.0475, energy_force_within_threshold: 0.0000, loss: 15.4902, epoch: 0.4963
forcesx_mae: 0.1098, forcesy_mae: 0.1340, forcesz_mae: 0.1413, forces_mae: 0.1284, forces_cos: 0.0480, forces_magnitude: 0.1877, ene

...

forcesx_mae: 0.0903, forcesy_mae: 0.1047, forcesz_mae: 0.1131, forces_mae: 0.1027, forces_cos: 0.0740, forces_magnitude: 0.1434, energy_mae: 2.8340, energy_force_within_threshold: 0.0000, loss: 13.0621, epoch: 0.9731
forcesx_mae: 0.1015, forcesy_mae: 0.1113, forcesz_mae: 0.1251, forces_mae: 0.1126, forces_cos: 0.0519, forces_magnitude: 0.1649, energy_mae: 2.8163, energy_force_within_threshold: 0.0000, loss: 14.0856, epoch: 0.9763
forcesx_mae: 0.0894, forcesy_mae: 0.1152, forcesz_mae: 0.1167, forces_mae: 0.1071, forces_cos: 0.0416, forces_magnitude: 0.1568, energy_mae: 3.3290, energy_force_within_threshold: 0.0000, loss: 14.1662, epoch: 0.9795
forcesx_mae: 0.0756, forcesy_mae: 0.0945, forcesz_mae: 0.0963, forces_mae: 0.0888, forces_cos: 0.0476, forces_magnitude: 0.1240, energy_mae: 2.5540, energy_force_within_threshold: 0.0000, loss: 11.4540, epoch: 0.9827
forcesx_mae: 0.0855, forcesy_mae: 0.1059, forcesz_mae: 0.1267, forces_mae: 0.1061, forces_cos: 0.0665, forces_magnitude: 0.1618, ene

device 0: 100%|██████████| 6250/6250 [02:01<00:00, 51.41it/s]

forcesx_mae: 0.0894, forcesy_mae: 0.1088, forcesz_mae: 0.1151, forces_mae: 0.1044, forces_cos: 0.0610, forces_magnitude: 0.1540, energy_mae: 3.2270, energy_force_within_threshold: 0.0000, loss: 13.7160, epoch: 1.0000
### Predicting on test.



device 0: 100%|██████████| 6250/6250 [01:53<00:00, 55.20it/s]


Writing results to results/2021-01-06-16-49-04-SchNet-example/s2ef_predictions.npz


### Load Checkpoint
Once training has completed a `Trainer` class, by default, is loaded with the best checkpoint as determined by training or validation (if available) metrics. To load a `Trainer` class directly with a pretrained model, specify the `checkpoint_path` as defined by your previously trained model (`checkpoint_dir` in cell 9):

In [20]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

pretrained_trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_trainer.load_pretrained(checkpoint_path="checkpoints/2021-01-06-16-49-04-SchNet-example/checkpoint.pt")

amp: false
cmd:
  checkpoint_dir: checkpoints/2021-01-06-17-23-12-SchNet-example
  identifier: SchNet-example
  logs_dir: logs/tensorboard/2021-01-06-17-23-12-SchNet-example
  print_every: 10
  results_dir: results/2021-01-06-17-23-12-SchNet-example
  seed: 0
  timestamp: 2021-01-06-17-23-12-SchNet-example
dataset:
  normalize_labels: false
  src: /home/jovyan/projects/ocp/data/s2ef/200k/train-demo
logger: tensorboard
logger_entity: null
logger_project: null
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
optim:
  batch_size: 16
  eval_batch_size: 8
  force_coefficient: 100
  lr_gamma: 0.1
  lr_initial: 0.0001
  lr_milestones:
  - 15
  - 20
  max_epochs: 1
  num_workers: 64
  warmup_epochs: 10
  warmup_factor: 0.2
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential e

True

## Predict

If a test has been provided in your config, predictions are generated and written to disk automatically upon training completion. Otherwise, to make predictions on unseen data a `torch.utils.data` DataLoader object must be constructed. Here we reference our test set to make predictions on. Predictions are saved in `{results_file}.npz` in your `results_dir`.

In [22]:
# make predictions on the existing test_loader
predictions = pretrained_trainer.predict(pretrained_trainer.test_loader, results_file="s2ef_results", disable_tqdm=False)

### Predicting on test.


device 0: 100%|██████████| 6250/6250 [01:53<00:00, 55.28it/s]


Writing results to results/2021-01-06-17-23-12-SchNet-example/s2ef_s2ef_results.npz


In [24]:
energies = predictions["energy"]
forces = predictions["forces"]